In [13]:
import pandas as pd
import numpy as np
from scipy.optimize import nnls
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from joblib import dump

In [14]:
df = pd.read_csv("./carros_training.csv", index_col=0).reset_index(drop=True)
print(df.shape)
df.head()

(6731, 66)


,chevrolet_aveo,chevrolet_beat,chevrolet_captiva sport,chevrolet_cavalier,chevrolet_chevy,chevrolet_cruze,chevrolet_equinox,chevrolet_malibu,chevrolet_sonic,chevrolet_spark,...,volkswagen_jetta,volkswagen_passat,volkswagen_polo,volkswagen_polo hatchback,volkswagen_tiguan,volkswagen_up!,volkswagen_vento,y_old,km,price
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,20,140000,33000
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,219143,35000
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,19,155000,39000
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,191775,42000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,19,165328,42000


In [15]:
df.describe()

,chevrolet_aveo,chevrolet_beat,chevrolet_captiva sport,chevrolet_cavalier,chevrolet_chevy,chevrolet_cruze,chevrolet_equinox,chevrolet_malibu,chevrolet_sonic,chevrolet_spark,...,volkswagen_jetta,volkswagen_passat,volkswagen_polo,volkswagen_polo hatchback,volkswagen_tiguan,volkswagen_up!,volkswagen_vento,y_old,km,price
count,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,...,6731.000000,6731.000000,6731.00000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6731.000000,6.731000e+03
mean,0.053187,0.011588,0.004903,0.009508,0.004903,0.016491,0.009657,0.008320,0.019759,0.014411,...,0.087654,0.016491,0.01144,0.010251,0.020502,0.005200,0.040856,4.556381,63132.429654,2.249850e+05
std,0.224422,0.107031,0.069853,0.097053,0.069853,0.127363,0.097801,0.090839,0.139183,0.119186,...,0.282812,0.127363,0.10635,0.100735,0.141721,0.071927,0.197971,3.140905,62893.539305,1.293082e+05
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.200000e+03
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,2.000000,29915.500000,1.459000e+05
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,4.000000,53940.000000,1.950000e+05
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,6.000000,82227.500000,2.650000e+05
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,20.000000,999999.000000,1.999000e+06


In [16]:
X = df.drop(['price'], axis=1)
Y = df.price
#Separar sets de entrenamiento y prueba
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = .10, random_state = 40)

In [17]:
#Evaluar resultados
def obtener_modelo(row):
    for col in row.index:
        if row[col] == 1:
            return col

def get_metrics(row, anunciado, prediccion, imprimir=False):
    carro = obtener_modelo(row)
    err = abs(prediccion - anunciado)/anunciado
    if imprimir:
        print(f"\nPara un {carro} con {row['km']}km y {row['y_old']} años de uso")
        print(f"Precio anunciado: {anunciado} Precio predicho: {prediccion}.")
        print(f"Error: {round(100*err,2)}%")
    return err

def evaluar_modelo(X_test, Y_test, modelo):
    errAcc = 0
    columnas = X_test.shape[0]
    for i in range(columnas):
        row = X_test.iloc[i]
        prediccion = int(np.dot(modelo, row))
        anunciado = Y_test.iloc[i]
        errAcc += get_metrics(row, anunciado, prediccion, i<5)
    errProm = round(100*errAcc/columnas, 2)
    print(f"\nScore: {100 - errProm}")
    print(f"Error promedio: {errProm}")

def evaluar_sk(X_test, Y_test, predicted):
    columnas = X_test.shape[0]
    for i in range(columnas):
        row = X_test.iloc[i]
        prediccion = int(predicted[i])
        anunciado = Y_test.iloc[i]
        get_metrics(row, anunciado, prediccion, i<5)


In [18]:
regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)
predicted = regr.predict(X_test)

print("Score:",regr.score(X_test, Y_test))
print("Error promedio:", round(100*(1 - regr.score(X_test, Y_test)), 2))
evaluar_sk(X_test, Y_test, predicted)
#Exportar archivo
dump(regr, './app/modelazo.joblib')

Score: 0.8103619187759896
Error promedio: 18.96

Para un chevrolet_aveo con 67445km y 5 años de uso
Precio anunciado: 129000 Precio predicho: 110996.
Error: 13.96%

Para un chevrolet_aveo con 28000km y 4 años de uso
Precio anunciado: 145000 Precio predicho: 138680.
Error: 4.36%

Para un volkswagen_up! con 17069km y 2 años de uso
Precio anunciado: 165000 Precio predicho: 175260.
Error: 6.22%

Para un nissan_sentra con 16110km y 4 años de uso
Precio anunciado: 175000 Precio predicho: 211940.
Error: 21.11%

Para un volkswagen_vento con 24940km y 1 años de uso
Precio anunciado: 255000 Precio predicho: 218033.
Error: 14.5%


['./app/modelazo.joblib']

In [19]:
modelo = nnls(X_train, Y_train)[0]
evaluar_modelo(X_test, Y_test, modelo)


Para un chevrolet_aveo con 67445km y 5 años de uso
Precio anunciado: 129000 Precio predicho: 133857.
Error: 3.77%

Para un chevrolet_aveo con 28000km y 4 años de uso
Precio anunciado: 145000 Precio predicho: 133857.
Error: 7.68%

Para un volkswagen_up! con 17069km y 2 años de uso
Precio anunciado: 165000 Precio predicho: 168456.
Error: 2.09%

Para un nissan_sentra con 16110km y 4 años de uso
Precio anunciado: 175000 Precio predicho: 179575.
Error: 2.61%

Para un volkswagen_vento con 24940km y 1 años de uso
Precio anunciado: 255000 Precio predicho: 181461.
Error: 28.84%

Score: 69.14
Error promedio: 30.86


In [20]:
pesos = []

for col in X_train:
    if col in 'km,y_old':
        continue
    indices = X_train[col] == 1
    dfX = X_train[indices][[col, 'km', 'y_old']]
    dfY = Y_train[indices]
    m = nnls(dfX, dfY)[0]
    pesos.append(m[0])

for elem in modelo[-2:]:
    pesos.append(elem)

modelo_chucky = np.array(pesos)
evaluar_modelo(X_test, Y_test, modelo_chucky)



Para un chevrolet_aveo con 67445km y 5 años de uso
Precio anunciado: 129000 Precio predicho: 133857.
Error: 3.77%

Para un chevrolet_aveo con 28000km y 4 años de uso
Precio anunciado: 145000 Precio predicho: 133857.
Error: 7.68%

Para un volkswagen_up! con 17069km y 2 años de uso
Precio anunciado: 165000 Precio predicho: 168456.
Error: 2.09%

Para un nissan_sentra con 16110km y 4 años de uso
Precio anunciado: 175000 Precio predicho: 179575.
Error: 2.61%

Para un volkswagen_vento con 24940km y 1 años de uso
Precio anunciado: 255000 Precio predicho: 181461.
Error: 28.84%

Score: 69.14
Error promedio: 30.86


In [21]:
#sgd = linear_model.SGDRegressor(fit_intercept=False, verbose=1)
#sgd.fit(X_train, Y_train)
#predicted = sgd.predict(X_test)

#evaluar_sk(X_test, Y_test, predicted)
#print("Score:",sgd.score(X_test, Y_test))
#print("Error promedio:", round(100*(1 - sgd.score(X_test, Y_test)), 2))

In [22]:
"""percep = linear_model.Perceptron(fit_intercept=False, verbose=1, max_iter=1000, n_jobs=1)
percep.fit(X_train, Y_train)"""

'percep = linear_model.Perceptron(fit_intercept=False, verbose=1, max_iter=1000, n_jobs=1)\npercep.fit(X_train, Y_train)'

In [23]:
"""predicted = percep.predict(X_test)
print("score:",percep.score(X_test, Y_test))
print(predicted)"""

'predicted = percep.predict(X_test)\nprint("score:",percep.score(X_test, Y_test))\nprint(predicted)'